In [1]:
from pipeline import *

(CVXPY) Jul 02 06:18:32 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 02 06:18:32 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier

np.random.seed(101)

In [3]:
df = pd.read_parquet(r'C:\Users\minht\Desktop\MLOps\data\phase-1\prob-2\raw_train.parquet')

In [4]:
df = df.sample(frac=1)

In [5]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,label
5995,V2,11.645719,V4,V4,2617.519333,V0,V3,4.0,V0,V0,...,V0,32.037193,none,V1,2.0,V2,2.0,yes,yes,0
17823,V1,24.351003,V4,V1,3665.757647,V0,V2,4.0,V2,V0,...,V2,41.756374,none,V1,1.0,V1,1.0,none,yes,0
33606,V0,17.826815,V4,V3,3989.534404,V0,V1,2.0,V2,V0,...,V2,22.011693,bank,V1,3.0,V3,2.0,yes,yes,0
48301,V3,24.964412,V2,V0,856.244244,V3,V4,4.0,V2,V0,...,V0,56.731349,none,V0,1.0,V2,2.0,none,yes,0
47869,V1,11.750023,V2,V0,1515.905359,V0,V3,2.0,V1,V0,...,V2,32.896634,none,V1,1.0,V2,1.0,yes,yes,1


In [6]:
train = df.drop(columns=['label'])
label = df['label']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state=101)

In [8]:
cats = ["feature1",
        "feature3",
        "feature4",
        "feature6",
        "feature7",
        "feature8",
        "feature9",
        "feature10",
        "feature11",
        "feature12",
        "feature14",
        "feature15",
        "feature16",
        "feature17",
        "feature19",
        "feature20"]

In [9]:
from sklearn.pipeline import FeatureUnion, Pipeline

In [10]:
columns = ['feature{}'.format(i) for i in range(1, 21)]

In [15]:
columns = ['feature1', 'feature2', 'feature5', 'feature7', 'feature13',
       'feature3', 'feature8', 'feature4', 'feature12', 'feature9',
       'feature11', 'feature14', 'feature6', 'feature15', 'feature17',
       'feature10', 'feature19']

cats = [i for i in cats if i in columns]

In [16]:
s = Pipeline(
    steps=(
        ('feature_select', FeatureSelector(columns)),
        ('woe', WOE(cats=cats, nums=[]))
    )
)

In [17]:
s.fit(X_train, y_train)

Pipeline(steps=[('feature_select',
                 FeatureSelector(_feature_names=['feature1', 'feature2',
                                                 'feature5', 'feature7',
                                                 'feature13', 'feature3',
                                                 'feature8', 'feature4',
                                                 'feature12', 'feature9',
                                                 'feature11', 'feature14',
                                                 'feature6', 'feature15',
                                                 'feature17', 'feature10',
                                                 'feature19'])),
                ('woe',
                 WOE(cats=['feature1', 'feature3', 'feature4', 'feature6',
                           'feature7', 'feature8', 'feature9', 'feature10',
                           'feature11', 'feature12', 'feature14', 'feature15',
                           'feature17', 'feature19'],
                     nums=[]))])

In [18]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\pipeline_2.pkl', 'wb') as f:
    pickle.dump(s, f)

In [19]:
X_train = s.transform(X_train)
X_val = s.transform(X_val)

C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = self.res[col].transform(X[col], metric='woe')
C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].apply(self._round)
C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [20]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.02,
    eval_metric='AUC'
)

In [21]:
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	test: 0.7827526	best: 0.7827526 (0)	total: 65.8ms	remaining: 2m 11s
1:	test: 0.7996551	best: 0.7996551 (1)	total: 72.5ms	remaining: 1m 12s
2:	test: 0.8047540	best: 0.8047540 (2)	total: 78.9ms	remaining: 52.5s
3:	test: 0.8051434	best: 0.8051434 (3)	total: 85ms	remaining: 42.4s
4:	test: 0.8042917	best: 0.8051434 (3)	total: 91.7ms	remaining: 36.6s
5:	test: 0.8049011	best: 0.8051434 (3)	total: 98.2ms	remaining: 32.6s
6:	test: 0.8066379	best: 0.8066379 (6)	total: 104ms	remaining: 29.7s
7:	test: 0.8068532	best: 0.8068532 (7)	total: 111ms	remaining: 27.6s
8:	test: 0.8069684	best: 0.8069684 (8)	total: 117ms	remaining: 25.9s
9:	test: 0.8097939	best: 0.8097939 (9)	total: 124ms	remaining: 24.6s
10:	test: 0.8113459	best: 0.8113459 (10)	total: 130ms	remaining: 23.5s
11:	test: 0.8108028	best: 0.8113459 (10)	total: 136ms	remaining: 22.6s
12:	test: 0.8110645	best: 0.8113459 (10)	total: 143ms	remaining: 21.8s
13:	test: 0.8115912	best: 0.8115912 (13)	total: 149ms	remaining: 21.1s
14:	test: 0.8117685	

122:	test: 0.8365886	best: 0.8365886 (122)	total: 869ms	remaining: 13.3s
123:	test: 0.8366613	best: 0.8366613 (123)	total: 877ms	remaining: 13.3s
124:	test: 0.8368746	best: 0.8368746 (124)	total: 885ms	remaining: 13.3s
125:	test: 0.8371790	best: 0.8371790 (125)	total: 892ms	remaining: 13.3s
126:	test: 0.8373413	best: 0.8373413 (126)	total: 900ms	remaining: 13.3s
127:	test: 0.8374197	best: 0.8374197 (127)	total: 905ms	remaining: 13.2s
128:	test: 0.8376523	best: 0.8376523 (128)	total: 913ms	remaining: 13.2s
129:	test: 0.8377549	best: 0.8377549 (129)	total: 920ms	remaining: 13.2s
130:	test: 0.8378775	best: 0.8378775 (130)	total: 927ms	remaining: 13.2s
131:	test: 0.8379158	best: 0.8379158 (131)	total: 933ms	remaining: 13.2s
132:	test: 0.8379674	best: 0.8379674 (132)	total: 939ms	remaining: 13.2s
133:	test: 0.8380139	best: 0.8380139 (133)	total: 947ms	remaining: 13.2s
134:	test: 0.8382433	best: 0.8382433 (134)	total: 953ms	remaining: 13.2s
135:	test: 0.8384630	best: 0.8384630 (135)	total: 9

238:	test: 0.8495849	best: 0.8495849 (238)	total: 1.67s	remaining: 12.3s
239:	test: 0.8496318	best: 0.8496318 (239)	total: 1.68s	remaining: 12.3s
240:	test: 0.8496803	best: 0.8496803 (240)	total: 1.69s	remaining: 12.3s
241:	test: 0.8496965	best: 0.8496965 (241)	total: 1.69s	remaining: 12.3s
242:	test: 0.8498119	best: 0.8498119 (242)	total: 1.7s	remaining: 12.3s
243:	test: 0.8498922	best: 0.8498922 (243)	total: 1.71s	remaining: 12.3s
244:	test: 0.8499551	best: 0.8499551 (244)	total: 1.71s	remaining: 12.3s
245:	test: 0.8501012	best: 0.8501012 (245)	total: 1.72s	remaining: 12.3s
246:	test: 0.8502535	best: 0.8502535 (246)	total: 1.73s	remaining: 12.3s
247:	test: 0.8504460	best: 0.8504460 (247)	total: 1.73s	remaining: 12.2s
248:	test: 0.8504578	best: 0.8504578 (248)	total: 1.74s	remaining: 12.2s
249:	test: 0.8504985	best: 0.8504985 (249)	total: 1.75s	remaining: 12.2s
250:	test: 0.8504878	best: 0.8504985 (249)	total: 1.75s	remaining: 12.2s
251:	test: 0.8507375	best: 0.8507375 (251)	total: 1.

360:	test: 0.8563957	best: 0.8563957 (360)	total: 2.48s	remaining: 11.3s
361:	test: 0.8564126	best: 0.8564126 (361)	total: 2.5s	remaining: 11.3s
362:	test: 0.8564038	best: 0.8564126 (361)	total: 2.51s	remaining: 11.3s
363:	test: 0.8564686	best: 0.8564686 (363)	total: 2.51s	remaining: 11.3s
364:	test: 0.8565249	best: 0.8565249 (364)	total: 2.52s	remaining: 11.3s
365:	test: 0.8565376	best: 0.8565376 (365)	total: 2.52s	remaining: 11.3s
366:	test: 0.8565343	best: 0.8565376 (365)	total: 2.53s	remaining: 11.3s
367:	test: 0.8566441	best: 0.8566441 (367)	total: 2.54s	remaining: 11.3s
368:	test: 0.8567837	best: 0.8567837 (368)	total: 2.54s	remaining: 11.3s
369:	test: 0.8568154	best: 0.8568154 (369)	total: 2.55s	remaining: 11.2s
370:	test: 0.8568186	best: 0.8568186 (370)	total: 2.56s	remaining: 11.2s
371:	test: 0.8568395	best: 0.8568395 (371)	total: 2.56s	remaining: 11.2s
372:	test: 0.8568327	best: 0.8568395 (371)	total: 2.57s	remaining: 11.2s
373:	test: 0.8568547	best: 0.8568547 (373)	total: 2.

482:	test: 0.8608722	best: 0.8608722 (482)	total: 3.29s	remaining: 10.3s
483:	test: 0.8608997	best: 0.8608997 (483)	total: 3.29s	remaining: 10.3s
484:	test: 0.8609801	best: 0.8609801 (484)	total: 3.3s	remaining: 10.3s
485:	test: 0.8610377	best: 0.8610377 (485)	total: 3.3s	remaining: 10.3s
486:	test: 0.8610340	best: 0.8610377 (485)	total: 3.31s	remaining: 10.3s
487:	test: 0.8610248	best: 0.8610377 (485)	total: 3.32s	remaining: 10.3s
488:	test: 0.8611907	best: 0.8611907 (488)	total: 3.32s	remaining: 10.3s
489:	test: 0.8612946	best: 0.8612946 (489)	total: 3.33s	remaining: 10.3s
490:	test: 0.8614577	best: 0.8614577 (490)	total: 3.33s	remaining: 10.2s
491:	test: 0.8614695	best: 0.8614695 (491)	total: 3.34s	remaining: 10.2s
492:	test: 0.8614620	best: 0.8614695 (491)	total: 3.35s	remaining: 10.2s
493:	test: 0.8614676	best: 0.8614695 (491)	total: 3.35s	remaining: 10.2s
494:	test: 0.8614712	best: 0.8614712 (494)	total: 3.36s	remaining: 10.2s
495:	test: 0.8615449	best: 0.8615449 (495)	total: 3.3

...

1446:	test: 0.8740978	best: 0.8741072 (1443)	total: 9.68s	remaining: 3.7s
1447:	test: 0.8740888	best: 0.8741072 (1443)	total: 9.69s	remaining: 3.69s
1448:	test: 0.8740832	best: 0.8741072 (1443)	total: 9.69s	remaining: 3.69s
1449:	test: 0.8740769	best: 0.8741072 (1443)	total: 9.7s	remaining: 3.68s
1450:	test: 0.8740841	best: 0.8741072 (1443)	total: 9.71s	remaining: 3.67s
1451:	test: 0.8740775	best: 0.8741072 (1443)	total: 9.71s	remaining: 3.67s
1452:	test: 0.8740814	best: 0.8741072 (1443)	total: 9.72s	remaining: 3.66s
1453:	test: 0.8740931	best: 0.8741072 (1443)	total: 9.73s	remaining: 3.65s
1454:	test: 0.8740962	best: 0.8741072 (1443)	total: 9.73s	remaining: 3.65s
1455:	test: 0.8740992	best: 0.8741072 (1443)	total: 9.74s	remaining: 3.64s
1456:	test: 0.8740943	best: 0.8741072 (1443)	total: 9.75s	remaining: 3.63s
1457:	test: 0.8740863	best: 0.8741072 (1443)	total: 9.75s	remaining: 3.63s
1458:	test: 0.8741099	best: 0.8741099 (1458)	total: 9.76s	remaining: 3.62s
1459:	test: 0.8741149	best:

1566:	test: 0.8741812	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.9s
1567:	test: 0.8741849	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.89s
1568:	test: 0.8741790	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.88s
1569:	test: 0.8741800	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.88s
1570:	test: 0.8741740	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.87s
1571:	test: 0.8741776	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.86s
1572:	test: 0.8741745	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.86s
1573:	test: 0.8741673	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.85s
1574:	test: 0.8741752	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.84s
1575:	test: 0.8741788	best: 0.8741967 (1524)	total: 10.5s	remaining: 2.84s
1576:	test: 0.8741784	best: 0.8741967 (1524)	total: 10.6s	remaining: 2.83s
1577:	test: 0.8741709	best: 0.8741967 (1524)	total: 10.6s	remaining: 2.82s
1578:	test: 0.8742225	best: 0.8742225 (1578)	total: 10.6s	remaining: 2.82s
1579:	test: 0.8742176	best

1682:	test: 0.8743325	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.13s
1683:	test: 0.8743375	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.12s
1684:	test: 0.8743273	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.11s
1685:	test: 0.8743235	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.1s
1686:	test: 0.8743302	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.1s
1687:	test: 0.8743220	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.09s
1688:	test: 0.8743195	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.08s
1689:	test: 0.8743205	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.08s
1690:	test: 0.8743184	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.07s
1691:	test: 0.8743132	best: 0.8743828 (1652)	total: 11.3s	remaining: 2.06s
1692:	test: 0.8743068	best: 0.8743828 (1652)	total: 11.4s	remaining: 2.06s
1693:	test: 0.8743186	best: 0.8743828 (1652)	total: 11.4s	remaining: 2.05s
1694:	test: 0.8743181	best: 0.8743828 (1652)	total: 11.4s	remaining: 2.04s
1695:	test: 0.8743183	best:

1803:	test: 0.8744229	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.31s
1804:	test: 0.8744194	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.3s
1805:	test: 0.8744194	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.3s
1806:	test: 0.8744172	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.29s
1807:	test: 0.8744172	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.28s
1808:	test: 0.8744217	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.28s
1809:	test: 0.8744205	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.27s
1810:	test: 0.8744271	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.26s
1811:	test: 0.8744138	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.26s
1812:	test: 0.8744134	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.25s
1813:	test: 0.8744043	best: 0.8745390 (1741)	total: 12.1s	remaining: 1.25s
1814:	test: 0.8744023	best: 0.8745390 (1741)	total: 12.2s	remaining: 1.24s
1815:	test: 0.8744059	best: 0.8745390 (1741)	total: 12.2s	remaining: 1.23s
1816:	test: 0.8744130	best:

1921:	test: 0.8745408	best: 0.8745473 (1920)	total: 12.9s	remaining: 523ms
1922:	test: 0.8745341	best: 0.8745473 (1920)	total: 12.9s	remaining: 516ms
1923:	test: 0.8745400	best: 0.8745473 (1920)	total: 12.9s	remaining: 509ms
1924:	test: 0.8745414	best: 0.8745473 (1920)	total: 12.9s	remaining: 503ms
1925:	test: 0.8745413	best: 0.8745473 (1920)	total: 12.9s	remaining: 496ms
1926:	test: 0.8745455	best: 0.8745473 (1920)	total: 12.9s	remaining: 489ms
1927:	test: 0.8745465	best: 0.8745473 (1920)	total: 12.9s	remaining: 482ms
1928:	test: 0.8745445	best: 0.8745473 (1920)	total: 12.9s	remaining: 476ms
1929:	test: 0.8745464	best: 0.8745473 (1920)	total: 12.9s	remaining: 469ms
1930:	test: 0.8745450	best: 0.8745473 (1920)	total: 12.9s	remaining: 462ms
1931:	test: 0.8745398	best: 0.8745473 (1920)	total: 12.9s	remaining: 456ms
1932:	test: 0.8745460	best: 0.8745473 (1920)	total: 13s	remaining: 449ms
1933:	test: 0.8745381	best: 0.8745473 (1920)	total: 13s	remaining: 442ms
1934:	test: 0.8745343	best: 0

In [22]:
vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
vl.columns = ['Feature', 'Importance']
vl = vl.sort_values('Importance', ascending=False)
vl['Feature'].values

array(['feature1', 'feature2', 'feature5', 'feature7', 'feature13',
       'feature3', 'feature8', 'feature4', 'feature12', 'feature9',
       'feature11', 'feature14', 'feature6', 'feature15', 'feature17',
       'feature10', 'feature19'], dtype=object)

In [23]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\problem2.pkl', 'wb') as f:
    pickle.dump(model, f)

In [24]:
tmp = model.predict_proba(X_train)[:, 1]
tmp = pd.DataFrame(tmp, columns=['Pro'])

In [25]:
tmp['Range'], _ = pd.qcut(tmp['Pro'], 10, retbins=True)

In [26]:
tmp['Range'].value_counts(normalize=True).sort_index()

(-0.000537, 0.0256]    0.1
(0.0256, 0.0511]       0.1
(0.0511, 0.0846]       0.1
(0.0846, 0.132]        0.1
(0.132, 0.199]         0.1
(0.199, 0.295]         0.1
(0.295, 0.425]         0.1
(0.425, 0.586]         0.1
(0.586, 0.767]         0.1
(0.767, 0.997]         0.1
Name: Range, dtype: float64

In [27]:
_

array([4.62596173e-04, 2.56203554e-02, 5.11177520e-02, 8.45635855e-02,
       1.31936916e-01, 1.98857889e-01, 2.94749266e-01, 4.25216823e-01,
       5.86057397e-01, 7.66938595e-01, 9.97276407e-01])